In [ ]:
### PIPELINING The saved model ####

In [1]:
# Important:: this model requires cuda, so at first install it using pip3 or conda as below:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
# or,   
#!conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia

# !pip install transformers torchaudio
!pip install sounddevice
!pip install wavio
!pip install scipy
!pip install soundfile


DEPRECATION: Loading egg at c:\users\shishir\miniconda3\envs\ilprl\lib\site-packages\huggingface_hub-0.16.4-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\users\shishir\miniconda3\envs\ilprl\lib\site-packages\huggingface_hub-0.16.4-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\users\shishir\miniconda3\envs\ilprl\lib\site-packages\huggingface_hub-0.16.4-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\users\shishir\miniconda3\envs\ilprl\lib\site-packages\huggingface_hub-0.16.4-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [2]:
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Tokenizer

model = Wav2Vec2ForCTC.from_pretrained("./ShishirAI_NepWav2Vec2")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("./ShishirAI_NepWav2Vec2")
# model.gradient_checkpointing_enable()

C:\Users\Shishir\miniconda3\envs\ILPRL\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Shishir\miniconda3\envs\ILPRL\Lib\site-packages\transformers\configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
C:\Users\Shishir\miniconda3\envs\ILPRL\L

In [3]:
import torch
import torchaudio

def preprocess_audio(audio_path, target_sample_rate=16000):
    waveform, sample_rate = torchaudio.load(audio_path)
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
    waveform = resampler(waveform)
    return waveform

def tokenize_audio(audio_path, tokenizer, target_sample_rate=16000):
    waveform = preprocess_audio(audio_path, target_sample_rate)
    input_values = tokenizer(waveform.squeeze().numpy(), return_tensors="pt").input_values
    return input_values




In [4]:
# Function for Predicting the transcription
def transcribe_audio(audio_path, model, tokenizer, target_sample_rate=16000):
    input_values = tokenize_audio(audio_path, tokenizer, target_sample_rate)
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription


In [7]:
# recoding for specified time : for e.g 8 secs

# # import required libraries
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
 
# Sampling frequency
freq = 44100
# freq = 16000
# Recording duration # change this duration as the button click condition
duration = 8
# Recording
recording = sd.rec(int(duration * freq),
                   samplerate=freq, channels=2, dtype='int16')
 
# Record audio for the given number of seconds
print("Recording in Progress")
sd.wait()
print("Recoding Done and saving the audio.")
 
# Convering numpy array to audio.
write("./Rec/file.wav", freq, recording)
print("Saved in REC folder")




Recording in Progress
Recoding Done and saving the audio.
Saved in REC folder


In [8]:
# # recording until the user presses enter.

# import sounddevice as sd
# import soundfile as sf

# # Global variables
# freq = 16000

# def record_audio(filename):
#     recording = sd.rec(int(10 * freq), samplerate=freq, channels=2, dtype='int16')
#     print("Recording in progress. Press Enter to stop recording.")
#     input()
#     sd.wait()
#     sf.write(filename, recording, freq)

# def main():
#     print("Press Enter to start recording...")
#     input()
#     print("Recording started. Press Enter again to stop recording.")
    
#     file_name = "./Rec/file.wav"
#     record_audio(file_name)
    
#     print(f"Recording saved as '{file_name}'.")

# if __name__ == "__main__":
#     main()


Press Enter to start recording...


Recording started. Press Enter again to stop recording.
Recording in progress. Press Enter to stop recording.


Recording saved as './Rec/file.wav'.


In [8]:
import torch
# audio_file_path = "./Rec/file.wav"
audio_file_path = "./Rec/file.wav"
transcription = transcribe_audio(audio_file_path, model, tokenizer)
print(transcription)


मेरो नाम शिसेर पौडेल हो मलाईबालचरलै गाइड गर्ड नु भएको थियो
